<a href="https://colab.research.google.com/github/GodHandOne/Deeplerning_Study/blob/Chapter_3/KerasAPI_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 了解Keras的核心API

## Dense层的实现

In [2]:
import tensorflow as tf
from tensorflow import keras

class SimpleDense(keras.layers.Layer):
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):  # build方法中创建权重
        input_dim = input_shape[-1]
        self.W = self.add_weight(shape=(input_dim, self.units),initializer="random_normal")
        self.b = self.add_weight(shape=(self.units,),initializer="zeros")

    def call(self, inputs): # call方法中定义前向传播计算过程
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

##对层进行实例化测试

In [9]:
my_dense=SimpleDense(units=32,activation=tf.nn.relu)
input_tensor=tf.ones(shape=(5,784))
print(input_tensor)
output_tensor=my_dense(input_tensor)
print(output_tensor)

tf.Tensor(
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]], shape=(5, 784), dtype=float32)
tf.Tensor(
[[0.         0.         1.6457102  1.4941249  0.         1.5837317
  0.         1.8391149  1.522424   0.         2.149601   1.6409099
  0.         0.5041622  3.2848248  3.3558016  0.         1.363205
  0.         2.0231903  0.6068451  2.2618003  0.6861208  0.59084666
  1.1703461  1.6164205  1.1740979  2.0147169  0.6788005  0.
  0.         1.1436468 ]
 [0.         0.         1.6457102  1.4941249  0.         1.5837317
  0.         1.8391149  1.522424   0.         2.149601   1.6409099
  0.         0.5041622  3.2848248  3.3558016  0.         1.363205
  0.         2.0231903  0.6068451  2.2618003  0.6861208  0.59084666
  1.1703461  1.6164205  1.1740979  2.0147169  0.6788005  0.
  0.         1.1436468 ]
 [0.         0.         1.6457102  1.4941249  0.         1.5837317
  0.         1.8391149  1.522424   0.         2

自动推断状态，动态构建层

层兼容性：每一层只接收特定形状的输入张量，并返回特定形状的输出张量

在实现层时，将前向传播放在call（）方法中

在Keras中构建模型通常的方法：

①直接作为Model类的子类

②使用函数式API，能够用更少的代码

##compile()方法

关键参数：

优化器、损失函数、评价指标

In [ ]:
from keras.engine.training import optimizer
model=keras.Sequential([keras.layers.Dense(1)]) # 线性分类器
model.compile(optimizer="rmsprop",loss="mean_squared_error",metrics=["accuracy"]) # 指定优化器的名称，损失函数的名称，指标列表

# model.compile(optimizer=keras.optimizers.RMSprop(),
# loss=keras.losses.MeanSquaredError(),
# metrics=[keras.metrics.BinaryAccuracy()]) 另一种书写方法，可以直接调用函数

##fit()方法，执行训练循环

关键参数：

需要训练的数据（包括输入和目标）、训练轮数（epoch）、每轮小批量梯度下降中使用的批量大小

In [ ]:
history = model.fit(
    inputs, #输入样本，一个NumPy数组
    targets,  #对应的训练目标，一个NumPy数组
    epochs=5,  #训练循环将对数据迭代5次
    batch_size=128  #训练循环的批量大小为128
)

要想查看模型在新数据上的性能，标准做法是保留训练数据的一个子集作为验证数据（validation data）。你不会在这部分数据上训练模型，但会用它来计算损失值和指标值。实现方法是在fit()中使用validation_data参数，和训练数据一样，验证数据也可以作为NumPy数组或TensorFlow Dataset对象传入。

In [ ]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])

indices_permutation = np.random.permutation(len(inputs))  # (本行及以下2行)为避免验证数据都来自同一个类别，需要使用随机索引排列将数据打乱
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

num_validation_samples = int(0.3 * len(inputs))  # (本行及以下4行)保留30%的训练数据用于验证（我们不会将这部分数据用于训练，而是保留下来用于计算验证损失和指标）
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]
model.fit(
    training_inputs,  # (本行及以下1行)训练数据，用于更新模型权重
    training_targets,
    epochs=5,
    batch_size=16,
    validation_data=(val_inputs, val_targets)  #验证数据，仅用来监控验证损失和指标
)

## 在训练完成后计算验证损失和指标，调用evaluate()方法

In [ ]:
loss_and_metrics = model.evaluate(val_inputs, val_targets, batch_size=128)

##推断
在训练好模型后来对新的数据进行预测

In [ ]:
#方法1
predictions = model(new_inputs)  # 接收一个NumPy数组或TensorFlow张量，返回一个TensorFlow张量
#方法2
predictions = model.predict(new_inputs, batch_size=128)  #接收一个NumPy数组或Dataset对象，返回一个NumPy数组